<a href="https://colab.research.google.com/github/sherirosalia/cms_19/blob/main/cms_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import date, datetime
import os, sys 
import io
import ee
import os



In [ ]:
## set up pandas dataframe using set_options to warn when we are working on a copy instead of ooriginal dataframe
pd.set_option('mode.chained_assignment','warn')

In [ ]:
# dataframe will display without being truncated
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 100)
# pd.set_option('display.width', 768)

In [ ]:
#mounting the google drive to access the files
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# quotechar='"', doublequote=True, quoting=csv.QUOTE_NONNUMERIC
phys_csv_path = "/content/drive/My Drive/cms_19/physicians.csv"

In [ ]:
# backup of clean dataframe, types will not be fully preserved so on import reassign, but easier with unconformity removed
# df = pd.read_csv(csv_path, dtype={'Jobs Saved':int, 'NAICS Code': str, 'NAICS Category': str, }, parse_dates=[10] )
pdf = pd.read_csv(phys_csv_path, index_col=None, header='infer',quotechar='"', )

pdf.sample(30)

In [ ]:
#sample regex to reference 
# re.sub('[^A-Za-z0-9]+', '', mystring)
# df.power.apply(lambda x: re.search(r'(\d+) ch', x).group(1))
# df['historical_rank_new']=df['historical_rank'].str.extract('(^[\d]{1,4})')
# df=df.str.extract('[^A-Za-z0-9]+')
# df

In [ ]:
#need to clean up extra characters
#import# 1086675: "6037,KAREN,,SIMON,,,,,,227 E 19TH ST,"8TH FLOOR, BLDG 'C'",NEW YORK,NY,10003-2602,UNITED STATES,,All..."

In [ ]:
problem=pdf.iloc[[1086672]]
problem

regex remove single quotes to make import into sql easier

In [ ]:
pdf['Physician_Profile_Address_Line_2']=pdf['Physician_Profile_Address_Line_2'].str.replace(r"[\"\',]", '')

In [ ]:
pdf['Physician_Profile_Address_Line_1']=pdf['Physician_Profile_Address_Line_1'].str.replace(r"[\"\',]", '')

In [ ]:
physid=pdf[pdf.Physician_Profile_Last_Name == 'SUMNICHT']
physid

In [ ]:
# boston.to_csv("physicians.csv", encoding="utf-8", index=False, )

In [ ]:
len(pdf)

In [ ]:
# pdf.columns.to_list()

In [ ]:
pdf.to_csv("/content/drive/My Drive/cms_19/physicians_clean.csv", encoding="utf-8", index=False, )



In [ ]:
own_csv_path="/content/drive/My Drive/cms_19/ownership.csv"

In [ ]:
# backup of clean dataframe, types will not be fully preserved so on import reassign, but easier with unconformity removed
# df = pd.read_csv(csv_path, dtype={'Jobs Saved':int, 'NAICS Code': str, 'NAICS Category': str, }, parse_dates=[10] )
own_df = pd.read_csv(own_csv_path, index_col=None, header='infer')



In [ ]:
# uncomment to see sample of 50 rows
# own_df.sample(50)

In [ ]:
#get a list of column names
own_df.columns.to_list()

clean up: 
remove extra characters such as single quotes and hashtags from 'Recipient_Primary_Business_Street_Address_Line1'
retain first 5 characters in zipcode


In [ ]:
own_df.Change_Type.value_counts()

In [ ]:
## old school python 'NEW'
counter=0
count_of_new=0
count_of_not_new=0
for i in own_df.Change_Type:
  
  if i == 'NEW':
    counter=counter+1
    
    count_of_new= count_of_new+1
  if i != 'NEW':
      counter=counter+1
      count_of_not_new= count_of_not_new+1
      
print(f'this is the count of not new {counter}')
print(f'this is the count of not new {count_of_new}')
print(f'this is the count of not new {count_of_not_new}')


In [ ]:
#zipcode trimmed to 5 characters
own_df['Recipient_Zip_Code'] =  own_df['Recipient_Zip_Code'].str.split('-').str[0]

In [ ]:
#uncomment to see sample of 10 rows
#own_df.sample(10)

In [ ]:
own_df.sample(20)

In [ ]:
#clean addresses of single quote
own_df['Recipient_Primary_Business_Street_Address_Line1']=own_df['Recipient_Primary_Business_Street_Address_Line1'].str.replace(r"[\"\',]", '')

In [ ]:
own_df['Recipient_Primary_Business_Street_Address_Line2']=own_df['Recipient_Primary_Business_Street_Address_Line2'].str.replace(r"[\"\',]", '')

In [ ]:
#error "NEW,122203,MATTHEW,ROSS,ALBERT,,661 E ALTAMONTE DR,SUITE 220,ALTAMONTE SPRINGS,FL,32701,United State...

In [ ]:
issue=own_df[own_df.Physician_Last_Name == 'ALBERT'] 
issue

In [ ]:
#save cleaned file for import into sql
own_df.to_csv("/content/drive/My Drive/cms_19/ownership_clean.csv", encoding="utf-8", index=False, )


In [ ]:
res_csv_path="/content/drive/My Drive/cms_19/research_payments.csv"

In [ ]:
res_df = pd.read_csv(res_csv_path, index_col=None, header='infer')


In [ ]:
res_df.sample(50)

In [ ]:
res_df.columns.to_list()

In [ ]:
gen_csv_path="/content/drive/My Drive/cms_19/general_payments.csv"

In [ ]:
gen_pay_df = pd.read_csv(gen_csv_path, index_col=None, dtype=str, header='infer', )


In [ ]:
gen_pay_df.sample(30)

In [ ]:
gen_pay_df.columns.to_list()

Clean up general payments file



In [ ]:
#zipcode trimmed to 5 characters
gen_pay_df['Recipient_Postal_Code'] =  gen_pay_df['Recipient_Postal_Code'].str.split('-').str[0]

In [ ]:
gen_pay_df['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name'].value_counts()

In [ ]:
gen_pay_df['Physician_Specialty'].value_counts()

In [ ]:
gen_pay_df['Physician_Specialty'].value_counts()


In [ ]:
vc = gen_pay_df['Physician_Specialty'].value_counts()


In [ ]:
#many categories, not sure what I want to do long term with this column
# vc[vc > 20000].index[0:20]
vc[vc > 2000] 

In [ ]:
gen_pay_df.describe()

In [ ]:
#sample regex to reference 
# re.sub('[^A-Za-z0-9]+', '', mystring)
# df.power.apply(lambda x: re.search(r'(\d+) ch', x).group(1))
# df['historical_rank_new']=df['historical_rank'].str.extract('(^[\d]{1,4})')
# df=df.str.extract('[^A-Za-z0-9]+')
# df

#Physician_Specialty